This is a concept notebook to investigate Glue unit conversion behavior integrated into Jdaviz spectrum viewer.

In [ ]:
import numpy as np
from astropy import units as u
from specutils import Spectrum1D

from jdaviz import Specviz

First spectrum.

In [ ]:
wave1 = np.linspace(2, 5, 10) * u.um
flux1 = [1, 2, 3, 4, 5, 5, 4, 3, 2, 1] * u.Jy
spec1 = Spectrum1D(flux=flux1, spectral_axis=wave1)

print(wave1)
print(flux1)

Second spectrum in different units and with slight offsets in spectral axis.

In [ ]:
wave2 = (wave1 + (0.1 * u.um)).to(u.GHz, u.spectral())
flux2 = flux1.to(u.mJy)
spec2 = Spectrum1D(flux=flux2, spectral_axis=wave2)

print(wave2)
print(wave2.to(u.um, u.spectral()))
print(flux2)

Fire up Specviz.

In [ ]:
specviz = Specviz()

In [ ]:
specviz.show()

Load the data into Specviz. Desired behavior:

1. "Jy_um" would load with Jy in Y-axis and um in X-axis.
2. "mJy_GHz" would load with data automatically converted to Jy and um in the plot. You would see the same shape but slightly offset in X-axis, just slightly.

In [ ]:
specviz.load_data(spec1, data_label="Jy_um")

In [ ]:
specviz.load_data(spec2, data_label="mJy_GHz")

Change the spectral axis display unit to GHz.

In [ ]:
viewer = specviz.app.get_viewer("spectrum-viewer")

In [ ]:
viewer.state.x_display_unit = "GHz"

Change the flux axis display unit to FLAM.

In [ ]:
FLAM = u.erg / (u.s * u.cm * u.cm * u.AA)

# If astropy can do it, Jdaviz should too.
spec1.flux.to(FLAM, u.spectral_density(spec1.spectral_axis))

In [ ]:
# this MIGHT fail depending on the version of astropy (since glue harcodes the expected string formatting 
# for units, whereas astropy recently changed the default order of units)
try:
    viewer.state.y_display_unit = FLAM.to_string()
except ValueError as e:
    print("setting y_display_unit failed: ", repr(e))

The plugin select component, however, is unit-aware and will handle mapping to the string formatting expected by glue

In [ ]:
uc = specviz.plugins['Unit Conversion']
uc.flux_unit.choices

In [ ]:
uc.flux_unit = FLAM.to_string()

Change the spectral axis again, this time to Angstrom via the plugin API.

In [ ]:
uc.spectral_unit = 'Angstrom'

Change everything back to original units.

In [ ]:
viewer.state.x_display_unit = "micron"
viewer.state.y_display_unit = "Jy"